Calculating cross correlations for all Emu sims I have. 

In [1]:
import numpy as np
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import TrivialPhaseSpace, Zheng07Cens
from halotools.empirical_models import NFWPhaseSpace, Zheng07Sats
from halotools.sim_manager import CachedHaloCatalog

In [3]:
class RedMagicCens(Zheng07Cens):
    """Slight tweak of the Zheng model to add a new parameter, f_c
    """
    #TODO what meaning does Luminosity threshold hold here?
    def __init__(self,**kwargs):
        
        upper_occupation_bound = 1.0
        super(RedMagicCens,self).__init__(**kwargs)

        if 'f_c' not in self.param_dict:
            self.param_dict['f_c'] = 0.19 #add in best fit of new param.
        
    def mean_occupation(self, **kwargs):
        "See Zheng07 for details"
        return self.param_dict['f_c']*super(RedMagicCens,self).mean_occupation(**kwargs)
    
class RedMagicSats(Zheng07Sats):
    "Slight tweak of Zheng model to add new parameter, f_c"
    
    def __init__(self, **kwargs):
        
        upper_occupation_bound = float("inf")
        super(RedMagicSats,self).__init__(**kwargs)
        
        #TODO not sure if this will work for the whole model; will need to test
        #may need to 'modulate'
        #if 'f_c' not in self.param_dict:
        #    self.param_dict['f_c'] = 0.19 #add in best fit of new param.
        
    def mean_occupation(self, **kwargs):
        "See Zheng07 for details"
        f_c = 1
        if 'f_c' in self.param_dict:
            f_c = self.param_dict['f_c']
        
        return super(RedMagicSats,self).mean_occupation(**kwargs)/f_c

In [4]:
from halotools.sim_manager import CachedHaloCatalog
from halotools.empirical_models import PrebuiltHodModelFactory
scale_factors = [0.25,0.333,0.5,  0.540541, 0.588235, 0.645161, 0.714286, 0.8, 0.909091, 1.0 ]
#scale_factors = [1.0]
halocats = {}
models = {}
for sf in scale_factors:
    rz = 1.0/sf-1
    halocats[sf] = CachedHaloCatalog(simname = 'emu', halo_finder = 'rockstar',version_name = 'most_recent',redshift = rz)
    #halocats[sf] = CachedHaloCatalog(simname = 'multidark', halo_finder = 'rockstar',redshift = rz)

    print len(halocats[sf].halo_table)
    
    models[sf] = HodModelFactory(
        centrals_occupation = RedMagicCens(redshift = rz),
        centrals_profile = TrivialPhaseSpace(redshift = rz),
        satellites_occupation = RedMagicSats(redshift = rz),
        satellites_profile = NFWPhaseSpace(redshift = rz))
    
    
    #models[sf] = PrebuiltHodModelFactory('Zheng07', redshift = rz)

1405119
4205502
8401414
8400370
8399650
9799696
9800916
9802571
9806644
9943398


In [12]:
pmass = 3.9876e10
npart = 30
for sf in scale_factors:
    made_cut = halocats[sf].halo_table['halo_mvir']<pmass*npart
    print round(1.0/sf-1, 2),'\t', len(halocats[sf].halo_table['halo_mvir'][made_cut])

3.0 	1073341
2.0 	2830023
1.0 	4888611
0.85 	4762121
0.7 	4639181
0.55 	5270345
0.4 	5107314
0.25 	4952729
0.1 	4791077
0.0 	4737811


In [13]:
for sf in scale_factors:
    models[sf].populate_mock(halocats[sf], Num_ptcl_requirement = npart)
    print round(1.0/sf-1, 2), '\t', len(models[sf].mock.halo_table)

3.0 	58
2.0 	111
1.0 	2093
0.85 	1824
0.7 	1531
0.55 	1621
0.4 	1458
0.25 	1344
0.1 	1402
0.0 	5205587


In [ ]:
from halotools.mock_observables import return_xyz_formatted_array, tpcf, tpcf_one_two_halo_decomp, wp
data = {}
pi_max = 40
rp_bins = np.logspace(-1,1.25,15)
rp_bin_centers = (rp_bins[:1] + rp_bins[1:])/2.

rbins = np.logspace(-1, 1.25, 15)
rbin_centers = (rbins[1:]+rbins[:-1])/2

for sf in scale_factors:
    x, y, z = [models[sf].mock.galaxy_table[c] for c in ['x','y','z'] ]
    pos = return_xyz_formatted_array(x,y,z)

    xi_all = tpcf(pos, rbins, period = models[sf].mock.Lbox, num_threads = 4)

    halo_hostid = models[sf].mock.galaxy_table['halo_id']

    xi_1h, xi_2h = tpcf_one_two_halo_decomp(pos,
                    halo_hostid, rbins, 
                    period = models[sf].mock.Lbox, num_threads = 4,
                    max_sample_size = 1e7)
    

    wp_all = wp(pos, rp_bins, pi_max, period=models[sf].mock.Lbox, num_threads = 4)
    
    data[sf] = (xi_all, xi_1h, xi_2h, wp_all)

/u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/halotools/mock_observables/clustering_helpers.py:64: UserWarning: 
 `sample1` exceeds `max_sample_size` 
downsampling `sample1`...
  warn(msg)


In [ ]:
from itertools import cycle
colors = cycle(sns.color_palette())
fig = plt.figure(figsize = (15, 15))

for sf, color in zip(scale_factors, colors):
    plt.subplot(211)
    rz = 1.0/sf -1 
    plt.plot(rbin_centers, data[sf][0],
             label='$z = %.2f$'%rz, color=color)
    plt.plot(rbin_centers, data[sf][1], ls = '--', color = color)
    plt.plot(rbin_centers, data[sf][2], ls = '-.', color = color)

    plt.subplot(221)
    plt.plot(rp_bin_centers, data[sf][3],
             label='$z = %.2f$'%rz,
             color= color )
    
plt.subplot(211)
plt.title('Cross Correlations')
plt.xlim(xmin = 0.1, xmax = 10)
plt.ylim(ymin = 1, ymax = 1e4)
plt.loglog()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\xi_{\rm gg}(r)$', fontsize=25)
plt.legend(loc='best', fontsize=20)

plt.subplot(221)
plt.title('Projected Cross Correlations')
plt.xlim(xmin = 0.1, xmax = 10)
plt.ylim(ymin = 0.5, ymax = 5e3)
plt.loglog()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r_{\rm p} $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$w_{\rm p}(r_{\rm p})$', fontsize=25)
plt.legend(loc='best', fontsize=20)